In [ ]:
from pywinauto import application
import time
import os

# 증권사 API 자동 실행
os.system('taskkill /IM coStarter* /F /T')
os.system('taskkill /IM CpStart* /F /T')
os.system('wmic process where "name like \'%coStarter%\'" call terminate')
os.system('wmic process where "name like \'%CpStart%\'" call terminate')
time.sleep(5)        

app = application.Application()
app.start('C:\CREON\STARTER\coStarter.exe /prj:cp /id:아이디 /pwd:비밀번호 /pwdcert:인증서 비밀번호 /autostart')
time.sleep(60)

서버가 정상적으로 접속되었습니다.


In [ ]:
import win32com.client

# 서버 접속 확인
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect

if (bConnect == 1): #접속 - 1, 비접속 - 0
    print("서버가 정상적으로 접속되었습니다.")
else:
    print("서버가 정상적으로 연결되지 않았습니다. ")
    exit()

서버가 정상적으로 접속되었습니다.


In [ ]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 종목명 -> 종목코드 변환
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
Stockname = input("종목명을 입력해주세요 : ")
Stockcode = instCpStockCode.NameToCode(Stockname)
tablename = Stockname

#now = time.strftime('%Y%m%d')

Stockpastday = 20200101
Stockpresentday = 20210203

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

# 관심종목 객체 구하기
objRq = win32com.client.Dispatch('CpSysDib.CpSvr7254')
objRq.SetInputValue(0, Stockcode)
objRq.SetInputValue(1, 0)  # 일자별
# (0:사용자지정 1:1개월, 2:2개월 3:3개월 4:6개월,5:최근5일 6:일별)
objRq.SetInputValue(2, Stockpastday) # 과거일자
objRq.SetInputValue(3, Stockpresentday) # 최신일자
objRq.SetInputValue(4, ord('0'))  # '0' 순매수 '1' 매매비중
objRq.SetInputValue(5, 0)  # '전체
objRq.SetInputValue(6, ord('1'))  # '1' 순매수량 '2' 추정금액(백만)
 
sumcnt = 0
df = None
df = pd.DataFrame(columns=('Date', 'close', '개인', '외국인', '기관계', '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '사모펀드', '기타법인', '기타외인'))

while True:
    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한
    
    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)

    objRq.BlockRequest()

    cnt = objRq.GetHeaderValue(1)
    sumcnt += cnt
 
    for i in range(cnt):
        item = {}
        item['Date'] = objRq.GetDataValue(0, i)
        item['close'] = objRq.GetDataValue(14, i)
        item['개인'] = objRq.GetDataValue(1, i)
        item['외국인'] = objRq.GetDataValue(2, i)
        item['기관계'] = objRq.GetDataValue(3, i)
        item['금융투자'] = objRq.GetDataValue(4, i)
        item['보험'] = objRq.GetDataValue(5, i)
        item['투신'] = objRq.GetDataValue(6, i)
        item['은행'] = objRq.GetDataValue(7, i)
        item['기타금융'] = objRq.GetDataValue(8, i)
        item['연기금'] = objRq.GetDataValue(9, i)
        item['사모펀드'] = objRq.GetDataValue(12, i)
        item['기타법인'] = objRq.GetDataValue(10, i)
        item['기타외인'] = objRq.GetDataValue(11, i)
 
        df.loc[len(df)] = item
 
    # 1000개 처리
    if sumcnt > 1000:
        break
    # 연속 처리
    if objRq.Continue != True:
        break

df['Date'] = df['Date'].astype('str')
df['Date'] = pd.to_datetime(df['Date'])

df1 = df
df = df.drop(['close'], axis=1)

In [ ]:
df

,Date,개인,외국인,기관계,금융투자,보험,투신,은행,기타금융,연기금,사모펀드,기타법인,기타외인
0,2021-02-03,195381,-98313,-83543,-18144,-11365,-27149,-1000,-7053,-40342,21510,-18405,4880
1,2021-02-02,230555,-110146,-103690,-24371,-11365,-29689,-1000,-7053,-50626,20414,-19099,2380
2,2021-02-01,178213,-100588,-59996,-21072,-10883,-7643,-1000,-7053,-32414,20069,-19497,1868
3,2021-01-29,180129,-93263,-69532,-29334,-10834,-8956,-1000,-7053,-32428,20073,-19197,1863
4,2021-01-28,157064,-71115,-68255,-29619,-10838,-7376,-1000,-7053,-32415,20046,-19464,1770
5,2021-01-27,114428,-29977,-65701,-28502,-10838,-6199,-1000,-7053,-32162,20053,-19964,1214
6,2021-01-26,18002,29983,-33174,-27665,-4180,-1509,-1000,-7053,-11211,19444,-15801,990
7,2021-01-25,-7625,66621,-48780,-28981,-4180,-19378,-1000,-7000,-7742,19501,-11160,944
8,2021-01-22,95414,-1441,-84583,-38111,-4634,-27550,-1000,-7000,-25722,19434,-10447,1057
9,2021-01-21,105283,-10345,-86626,-36054,-4618,-30876,-1000,-7000,-25482,18404,-9304,992


In [ ]:
import plotly.express as px

fig = px.line(df, x="Date", y=df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title='투자주체별현황')

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.show()

In [ ]:
df1 = df1.drop(['Date'], axis=1)
df1 = df1.astype(float)

#df1.dtypes

# df1['close'].corr(df1['기관계'])
corr = df1.corr(method='pearson')
print(Stockname,'투자주체별 상관계수')
print(corr['close'].sort_values(ascending=False))

이녹스첨단소재 투자주체별 상관계수
close    1.000000
외국인      0.781425
연기금      0.520766
보험       0.309904
기관계      0.128324
사모펀드     0.093567
기타법인     0.082618
기타금융    -0.150244
기타외인    -0.169479
투신      -0.171305
은행      -0.240890
금융투자    -0.369004
개인      -0.659241
Name: close, dtype: float64
